In [1]:
from pyspark.sql.types import StructField, StructType, StringType, LongType, IntegerType, StringType, FloatType, DataType, BooleanType, ArrayType, DoubleType

# A Spark Session is how we interact with Spark SQL to create Dataframes
from pyspark.sql import SparkSession

# This will help catch some PySpark errors
from py4j.protocol import Py4JJavaError

from pyspark.sql.functions import * # isnan, when, count, col, monotonically_increasing_id, udf 

from pyspark.ml.feature import PCA, VectorAssembler
from pyspark.ml.linalg import Vectors

import datetime 

import time


spark = (SparkSession.builder.master('yarn')
.appName('my_app').
.config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest.jar')
.config('spark.scheduler.mode','FAIR')
.config('spark.sql.cbo.enabled',True)
.config('spark.sql.cbo.joinReorder.enabled',True)
.config('spark.executer.instances',3)#8
.config('spark.executer.cores',4)
.config('spark.driver.cores',4)
.config('spark.executer.memory','4g')
.config('spark.driver.memory','4g')
.config('spark.yarn.executor.memoryOverhead','4g')
.config('spark.yarn.driver.memoryOverhead','4g')
.config("sprak.yarn.queue",'root.high')
.getOrCreate())

In [2]:
spark.sparkContext.getConf().getAll()

[(u'spark.app.id', u'application_1588908927341_0002'),
 (u'spark.eventLog.enabled', u'true'),
 (u'spark.dynamicAllocation.minExecutors', u'1'),
 (u'spark.driver.maxResultSize', u'123008m'),
 (u'spark.yarn.dist.jars',
  u'gs://spark-lib/bigquery/spark-bigquery-latest.jar'),
 (u'spark.ui.proxyBase', u'/proxy/application_1588908927341_0002'),
 (u'spark.driver.appUIAddress',
  u'http://cluster-tony1-m.c.umg-ds-jackalope.internal:4041'),
 (u'spark.yarn.am.memory', u'640m'),
 (u'spark.driver.port', u'33143'),
 (u'spark.executorEnv.PYTHONPATH',
  u'{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.7-src.zip'),
 (u'spark.driver.memory', u'500G'),
 (u'spark.app.name', u'my_app'),
 (u'spark.executor.instances', u'2'),
 (u'spark.jars', u'gs://spark-lib/bigquery/spark-bigquery-latest.jar'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  u'cluster-tony1-m'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.ui.f

In [ ]:
# print(table_df.printSchema()) The schema for the data frame I am using

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- user_country_code: string (nullable = true)
 |-- target_variable: long (nullable = true)
 |-- r2_genre_desc: string (nullable = false)
 |-- has_canopy_genre: long (nullable = true)
 |-- apple_genre: string (nullable = false)
 |-- has_apple_genre: long (nullable = true)
 |-- final_track_genre: string (nullable = false)
 |-- days_since_release: long (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- num_markets: long (nullable = true)
 |-- key: long (nullable = false)
 |-- mode: long (nullable = false)
 |-- tempo: double (nullable = false)
 |-- time_signature: long (nullable = false)
 |-- stream_duration: long (nullable = true)
 |-- stream_country_code: string (nullable = true)
 |-- consumer_group_detail: string (nullable = true)
 |-- num_artists: long (nullable = false)
 |-- artist_begin_year: long (nullable = false)
 |-- artist_gender: string (nullable = false)
 |-- artist_geographic_ar

In [15]:
# print(table_df.count()) 
# print(len(table_df.columns))

30976533
146


In [ ]:
# Train test split
st = time.time()
train, test = table_df.randomSplit([0.8, 0.2], seed = 2018)
print(time.time() - st)

0.657861948013


In [10]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.classification import RandomForestClassificationModel
import time 


from itertools import chain
import pandas as pd
import numpy as np

from sklearn.metrics import auc
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier
from pyspark.mllib.evaluation import MulticlassMetrics
from sklearn.metrics import roc_auc_score


import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import auc
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [ ]:
# This was what I used to train the model
# rf = RandomForestClassifier(labelCol="label", featuresCol="features",maxDepth=27, numTrees=100) 

# st = time.time()
# rfModel_27 = rf.fit(train)
# print(time.time() - st)

# ## get predictions on test set 
# st = time.time()
# predictions_27 = rfModel_27.transform(test) 
# print(time.time() - st) 

#rfModel_27.save('gs://some_bucket/rfModel_27')

In [11]:
# load the pretrained random forest model 
rfModel_27 = RandomForestClassificationModel.load('gs://some_bucket/rfModel_27')  

In [12]:
predictions_27 = rfModel_27.transform(test) 

In [ ]:
predictions_27.cache()

In [13]:
# get rdd of predictions
scoreAndLabels = predictions_27.select('label','probability').rdd.map(lambda row: (float(row['probability'][1]), float(row['label'])))


Py4JJavaError: An error occurred while calling o8860.javaToPython.
: java.lang.OutOfMemoryError: Requested array size exceeds VM limit
	at java.util.Arrays.copyOf(Arrays.java:3236)
	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:118)
	at java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:93)
	at java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:153)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1877)
	at java.io.ObjectOutputStream$BlockDataOutputStream.setBlockDataMode(ObjectOutputStream.java:1786)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1189)
	at java.io.ObjectOutputStream.writeObject(ObjectOutputStream.java:348)
	at org.apache.spark.serializer.JavaSerializationStream.writeObject(JavaSerializer.scala:43)
	at org.apache.spark.serializer.JavaSerializerInstance.serialize(JavaSerializer.